In [1]:

from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [2]:
!pip install deeppavlov
!pip install transformers
!pip install finbert-embedding
!pip install spacy
!pip install xgboost
!pip install sister
!pip install fuzzywuzzy
!pip install spacy[transformers]
!python -m spacy download en_core_web_trf


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 7.5 MB/s 
     |████████████████████████████████| 55 kB 4.5 MB/s 
     |████████████████████████████████| 145 kB 73.7 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 54 kB 3.3 MB/s 
     |████████████████████████████████| 57 kB 3.0 MB/s 
     |████████████████████████████████| 59 kB 9.0 MB/s 
     |████████████████████████████████| 8.2 MB 50.2 MB/s 
     |████████████████████████████████| 63 kB 2.5 MB/s 
     |████████████████████████████████| 80 kB 10.0 MB/s 
     |████████████████████████████████| 58 kB 6.8 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=46d6bfb8ba7981be0612ce6db765e3b689d9554bb56a34598a64c17caaa5d823
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built

In [1]:
%cd '/content/gdrive/MyDrive/Capstone/t2s'
from utilities import *

/content/gdrive/MyDrive/Capstone/t2s


2022-11-30 07:42:25.898 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/kbqa/wikidata/query_prediction_eng.tar.gz download because of matching hashes
INFO:deeppavlov.download:Skipped http://files.deeppavlov.ai/kbqa/wikidata/query_prediction_eng.tar.gz download because of matching hashes
2022-11-30 07:42:26.841 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/kbqa/wikidata/query_prediction_eng.pickle download because of matching hashes
INFO:deeppavlov.download:Skipped http://files.deeppavlov.ai/kbqa/wikidata/query_prediction_eng.pickle download because of matching hashes
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at haisongzhang/roberta-tiny-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.7/dist-pac

In [7]:
def text2sparql(query,ontology):

  #if
  sparql_query_list=[]

  template_id,n_entities, query_templates=template_classification(query)#
  print(query_templates)

  if template_id==4 or template_id==2:
    #print(get_single_relation(query,ontology))
    relation,relation_class=get_single_relation(query,ontology)#
    print(relation)

    entity,entity_class=get_single_entity(query,ontology)#
    query_validity=check_query_validity(entity, relation, ontology)


    if (entity!="Not found" and relation!="Not found" ):
      if query_validity==True:  
        for query_template in query_templates:
          sparql_query_list.append(query_template.replace("ENTITY_CLASS",entity_class.replace(" ","_")).replace("ENTITY",entity.replace(" ","_")).replace("RELATION_CLASS",relation_class.replace(" ","_")).replace("RELATION",relation.replace(" ","_")))
      else: 
        q="SELECT DISTINCT ?p ?o WHERE { <http://crypto.org/ENTITY_CLASS/ENTITY> ?p ?o.}"
        sparql_query_list.append(q.replace("ENTITY_CLASS",entity_class.replace(" ","_")).replace("ENTITY",entity.replace(" ","_")))

    else:
      sparql_query_list.append("Not found")

  if template_id==1:
    print("hello")
    entities, entities_class=get_entities(query,ontology)

    if len(entities)!=0:
      for i,entity in enumerate(entities):
        entity_class=entities_class[i]
        #print(entity, entity_class)
        sparql_query_list.append(query_templates.replace("ENTITY_CLASS",entity_class.replace(" ","_")).replace("ENTITY",entity.replace(" ","_")))
    else:
      sparql_query_list.append("Not found")

  if template_id==5:
        relation,relation_class=get_single_relation(query,ontology)#
        entities, entities_class=get_entities(query,ontology)
        aggregation=get_aggregation_method(query)
        #print(aggregation)
        if (relation!="Not found" and relation!="Not found"):
          for i,entity in enumerate(entities):
            query_validity=check_query_validity(entity, relation, ontology)
            entity_class=entities_class[i]

            if query_validity==True:              
              for agg_method in aggregation:
                #print(agg_method)
                sparql_query_list.append(query_templates[0].replace("ENTITY_CLASS",entity_class.replace(" ","_")).replace("ENTITY",entity.replace(" ","_")).replace("RELATION_CLASS",relation_class.replace(" ","_")).replace("RELATION",relation.replace(" ","_")).replace("AGGREGATION_METHOD",agg_method.replace(" ","_")))
            else:
                q= "SELECT DISTINCT ?p ?o WHERE { <http://crypto.org/ENTITY_CLASS/ENTITY> ?p ?o.}"
                sparql_query_list.append(q.replace("ENTITY_CLASS",entity_class.replace(" ","_")).replace("ENTITY",entity.replace(" ","_")))
     
        else:
          sparql_query_list.append("Not found")
  
  
  return sparql_query_list


query1="What did Ramit Sawhney author?"
query2="How many papers has Ramit Sawhney authored?"
query3="List the papers written by Ramit Sawhney"
query4="Which cryptocurrency costs the most?"
query6="Which cryptocurrency is costs the most?"
query7="Who is author of Cryptocurrency Bubble Detection?"

text2sparql(query4,"stanford")

['SELECT ?o WHERE { <http://crypto.org/ENTITY_CLASS/ENTITY> <http://crypto.org/RELATION_CLASS/RELATION> ?o. } ORDER BY AGGREGATION_METHOD(?o) LIMIT 1']


/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


['SELECT DISTINCT ?p ?o WHERE { <http://crypto.org/CRYPTOGRAPHY/crypto> ?p ?o.}',
 'SELECT DISTINCT ?p ?o WHERE { <http://crypto.org/DERIVATIVE/currency_market> ?p ?o.}']